**Bert预训练第一种**

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer

In [3]:
# 读取数据集
df = pd.read_csv('/content/sample_data/data.csv')
df.head()

,text,label,score
0,趵突泉管好你自己,1,0.646242
1,防李清照的围挡哈哈哈哈,2,0.909193
2,大青岛,2,0.987700
3,扑棱鹅子,2,0.561283
4,轩轩的家乡,2,0.999437


In [4]:
# 分割训练集和测试集
train_texts, test_texts, train_labels, test_labels = train_test_split(df['text'].to_list(), df['label'].to_list(), test_size=0.2)

In [5]:
# 加载BERT分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [6]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=128,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label)
        }

In [7]:
# 创建训练集和测试集的数据集实例
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
test_dataset = SentimentDataset(test_texts, test_labels, tokenizer)

In [8]:
from transformers import BertForSequenceClassification

In [9]:
# 加载BERT情感分类模型
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=3)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [10]:
# 定义训练参数
train_args = {
    'epochs': 2,
    'batch_size': 16,
    'learning_rate': 0.001,
    'eps': 1e-8,
    'weight_decay': 0.01
}

In [11]:
# 定义优化器和损失函数
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

In [12]:
# 定义训练函数
def train_fn(model, dataloader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    train_acc = 0
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        label = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=label)
        loss = criterion(outputs.logits, label)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += (outputs.logits.argmax(-1) == label).sum().item()

    train_loss /= len(dataloader)
    train_acc /= len(dataloader.dataset)

    return train_loss, train_acc



In [13]:
# 定义测试函数
def test_fn(model, dataloader, criterion, device):
    model.eval()
    test_loss = 0
    test_acc = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            label = batch['label'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=label)
            loss = criterion(outputs.logits, label)

            test_loss += loss.item()
            test_acc += (outputs.logits.argmax(-1) == label).sum().item()

    test_loss /= len(dataloader)
    test_acc /= len(dataloader.dataset)

    return test_loss, test_acc


In [14]:
# 将数据集转换为dataloader
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=train_args['batch_size'], shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=train_args['batch_size'])


In [15]:
!nvidia-smi

Fri Apr 28 02:31:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
model.to(device)
# 将模型移动到GPU上（如果可用）
#device = torch.device('cuda:0')
#model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [18]:
def predict_sentiment(text, model, tokenizer, device):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    ).to(device)

    output = model(encoding['input_ids'], encoding['attention_mask'])
    label = output.logits.argmax(-1).item()

    id2label = {v: k for k, v in label2id.items()}
    return id2label[label]

In [19]:
# 在测试集上进行预测
test_predictions = []
for batch in test_dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)

    outputs = model(input_ids, attention_mask=attention_mask)
    predictions = outputs.logits.argmax(-1).cpu().tolist()
    test_predictions.extend(predictions)

In [20]:
np_pre= np.array(test_predictions)
np_labels= np.array(test_labels)

In [21]:
# 计算测试集的准确率
test_accuracy = (np_pre==np_labels).mean()
print(f'Test Accuracy: {test_accuracy:.4f}')


Test Accuracy: 0.6394
